In [1]:
import evalml
from evalml import *
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from agents.tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from agents.data_agent import *
from agents.stock_agent import *
from backtest import *
from bitstamp import *
from model_winner_select import *


from sklearn.utils import shuffle

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

step = 1


In [3]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = train_keys,
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list,
                 verbose = False
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [4]:
load_from_disk = True
tec = TecAn(windows = 20, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(3, sourceDataGenerator, load_from_disk, 10)

In [5]:
trainX_raw, trainY_raw = online.load_train_data()


x, y = prepare_train_data(trainX_raw, trainY_raw, step)


print(Counter(y))

Counter({0: 25444, 1: 3356})


In [6]:
currency = "ethusd"

trainX_raw, trainY_raw, prices = online.load_val_data(currency)
val_x, val_y = prepare_train_data(trainX_raw, trainY_raw, step)

print(Counter(val_y))

Counter({0: 12323, 1: 2078})


In [7]:
from tpot.config import classifier_config_dict
# add FeatureSetSelector into tpot configuration
classifier_config_dict['tpot.builtins.FeatureSetSelector'] = {
    'subset_list': ['https://raw.githubusercontent.com/EpistasisLab/tpot/master/tests/subset_test.csv'],
    'sel_subset': [0,1] # select only one feature set, a list of index of subset in the list above
    #'sel_subset': list(combinations(range(3), 2)) # select two feature sets
}



In [30]:
%%time

from tpot import TPOTClassifier

model = TPOTClassifier(#generations=30, 
                       #population_size=50,
                       max_time_mins=24*60,
                       scoring='balanced_accuracy', 
                       verbosity=2, 
                       #template='Selector-Transformer-Classifier',
                       #config_dict=classifier_config_dict,
                       random_state=1,
                       config_dict='TPOT NN',
                       n_jobs=-1)


CPU times: user 24 µs, sys: 5 µs, total: 29 µs
Wall time: 36 µs


In [31]:
model.fit(x, y)

Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.6412857188301825

Generation 2 - Current best internal CV score: 0.6412857188301825

Generation 3 - Current best internal CV score: 0.6435163297107105

Generation 4 - Current best internal CV score: 0.6435163297107105

Generation 5 - Current best internal CV score: 0.6435163297107105

Generation 6 - Current best internal CV score: 0.6435163297107105

Generation 7 - Current best internal CV score: 0.6435163297107105

Generation 8 - Current best internal CV score: 0.6435163297107105

Generation 9 - Current best internal CV score: 0.6436229962970461

Generation 10 - Current best internal CV score: 0.6437278707204819

Generation 11 - Current best internal CV score: 0.6439398772452746

Generation 12 - Current best internal CV score: 0.6440182551734815

Generation 13 - Current best internal CV score: 0.6450305493036866

Generation 14 - Current best internal CV score: 0.6460362401183682

Generation 15 - Current best internal CV score: 0.646602

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd078285a60>
Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1295, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'



Generation 38 - Current best internal CV score: 0.6507831830321458

Generation 39 - Current best internal CV score: 0.6507831830321458

Generation 40 - Current best internal CV score: 0.6507831830321458

Generation 41 - Current best internal CV score: 0.6507831830321458

Generation 42 - Current best internal CV score: 0.6507831830321458

Generation 43 - Current best internal CV score: 0.6507831830321458

Generation 44 - Current best internal CV score: 0.6507831830321458

Generation 45 - Current best internal CV score: 0.6507831830321458

Generation 46 - Current best internal CV score: 0.6508235005560583

Generation 47 - Current best internal CV score: 0.6508235005560583

Generation 48 - Current best internal CV score: 0.6508235005560583

Generation 49 - Current best internal CV score: 0.6508235005560583

Generation 50 - Current best internal CV score: 0.6508235005560583

Generation 51 - Current best internal CV score: 0.6521442746018213


Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/multiprocessing/synchronize.py", line 88, in _cleanup
    unregister(name, "semaphore")
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/multiprocessing/resource_tracker.py", line 151, in unregister
    self._send('UNREGISTER', name, rtype)
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/multiprocessing/resource_tracker.py", line 160, in _send
    nbytes = os.write(self._fd, msg)
stopit.utils.TimeoutException: 



Generation 52 - Current best internal CV score: 0.6521442746018213

Generation 53 - Current best internal CV score: 0.6522738813072515

Generation 54 - Current best internal CV score: 0.6522738813072515

Generation 55 - Current best internal CV score: 0.6522738813072515

Generation 56 - Current best internal CV score: 0.6522738813072515

Generation 57 - Current best internal CV score: 0.6522738813072515

Generation 58 - Current best internal CV score: 0.6522738813072515

Generation 59 - Current best internal CV score: 0.6522738813072515

Generation 60 - Current best internal CV score: 0.6522738813072515

1442.10 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: BernoulliNB(SGDClassifier(CombineDFs(RobustScaler(ZeroCount(SelectPercentile(GaussianNB(input_matrix), percentile=45))), PCA(input_matrix, iterated_power=8, svd_solver=randomized)), alpha=0.001, eta0=1.0, fi

TPOTClassifier(config_dict='TPOT NN', max_time_mins=1440, n_jobs=-1,
               random_state=1, scoring='balanced_accuracy', verbosity=2)

In [32]:
model.export('tpot_boston_pipeline.py')

In [33]:
print(model.score(val_x, val_y))

0.5818098617130796


In [34]:
cache = CacheProvider(
        currency_list=[currency],
        verbose = False
    )

In [36]:
val_keys = [currency]
for key in val_keys:
    print(key)
    back, metric = eval_model(model = model,
                      hot_load_total = 100,
                      currency = key, 
                      cache = cache,
                      stop_loss = -1,
                      provider = online, step = step, verbose = False)
    back.report()
    print(metric)

ethusd
-11.53863% -> 88.46136529441762
Positive: 195(0.14301025641025641) Negative: 83(-0.47500722891566266)
{'recall': 0.6447368421052632, 'precision': 0.15076923076923077, 'f1': 0.24438902743142144, 'accuracy': 0.5455, 'roc_auc': 0.5887341095402163}


In [ ]:
automl = AutoMLSearch(X_train=x, y_train=y, problem_type='binary', 
                      max_batches=4,
                      objective='f1',
                      #additional_objectives=['auc', 'f1', 'precision'],
                      #additional_objectives=[ 'precision'],
                      ensembling=False)
automl.search()

In [ ]:
#automl.rankings

In [ ]:
pipeline = automl.best_pipeline

#pipeline.graph()

eval_data(pipeline, val_x, val_y)


In [ ]:
preds = pipeline.predict(val_x)

print(Counter(val_y))
print(Counter(preds))

In [ ]:
evalml.objectives.utils.get_core_objective_names()

In [ ]:
back = test_model(pipeline, "btcusd", online, step, verbose = True)

In [ ]:
back.report()